In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("FPGrowth").getOrCreate()

23/12/28 10:09:12 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/28 10:09:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/28 10:09:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.fpm import FPGrowth

In [3]:
from pyspark.sql.functions import split

transacoes = spark.read.text("/home/tkroza/PycharmProjects/ML_with_pyspark/data/Transacoes.txt").select(split("value","\s+").alias("transacoes"))
transacoes.show(truncate=False)

+-------------------------+
|transacoes               |
+-------------------------+
|[Cerveja, Pizza, Sorvete]|
|[Pizza, Sorvete]         |
|[Cerveja, Pizza]         |
|[Cerveja, Pizza, Sorvete]|
|[Cerveja, Pizza]         |
|[Pizza]                  |
+-------------------------+



In [4]:
fp = FPGrowth(itemsCol="transacoes", minSupport=0.5, minConfidence=0.5)
modelo = fp.fit(transacoes)
modelo.freqItemsets.show()

+----------------+----+
|           items|freq|
+----------------+----+
|       [Sorvete]|   3|
|[Sorvete, Pizza]|   3|
|       [Cerveja]|   4|
|[Cerveja, Pizza]|   4|
|         [Pizza]|   6|
+----------------+----+



In [5]:
modelo.associationRules.show()

+----------+----------+------------------+----+------------------+
|antecedent|consequent|        confidence|lift|           support|
+----------+----------+------------------+----+------------------+
| [Cerveja]|   [Pizza]|               1.0| 1.0|0.6666666666666666|
| [Sorvete]|   [Pizza]|               1.0| 1.0|               0.5|
|   [Pizza]| [Sorvete]|               0.5| 1.0|               0.5|
|   [Pizza]| [Cerveja]|0.6666666666666666| 1.0|0.6666666666666666|
+----------+----------+------------------+----+------------------+



In [6]:
modelo.transform(transacoes).show(truncate=False)

+-------------------------+------------------+
|transacoes               |prediction        |
+-------------------------+------------------+
|[Cerveja, Pizza, Sorvete]|[]                |
|[Pizza, Sorvete]         |[Cerveja]         |
|[Cerveja, Pizza]         |[Sorvete]         |
|[Cerveja, Pizza, Sorvete]|[]                |
|[Cerveja, Pizza]         |[Sorvete]         |
|[Pizza]                  |[Sorvete, Cerveja]|
+-------------------------+------------------+

